## MobileNetV2 (Image Classification, Lightweight)

In [3]:
import torch
import torchvision
from torchvision import transforms
import numpy as np
from tvm import relay


mobilenet = torchvision.models.mobilenet_v2(pretrained=True).eval()
input_shape = [1, 3, 224, 224]
x = torch.randn(input_shape)

y = mobilenet(x)
print("MobileNetV2 output shape:", y.shape)
scripted_mobilenet = torch.jit.trace(mobilenet, x).eval()

# Convert to TVM
input_name = "input0"
shape_list = [(input_name, x.shape)]
mod_mobilenet, params_mobilenet = relay.frontend.from_pytorch(scripted_mobilenet, shape_list)

print("✅ MobileNetV2 successfully converted to Relay")



/home/vscode/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/vscode/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MobileNetV2 output shape: torch.Size([1, 1000])
✅ MobileNetV2 successfully converted to Relay


## Compile Mobilenet

In [4]:
import tvm
from tvm.contrib import graph_executor

# Choose your target architecture
# Example: x86 CPU
target_str = "llvm -mtriple=x86_64-linux-gnu -mcpu=haswell"
target = tvm.target.Target(target_str)
dev = tvm.cpu(0)


print(f"Compiling MobileNetV2...")
with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod_mobilenet, target=target, params=params_mobilenet)
# Save compiled library
lib_name = f"MobileNetV2_tvm.so"
lib.export_library(lib_name)
print(f"✅ MobileNetV2 compiled and exported as {lib_name}")


Compiling MobileNetV2...


One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.


✅ MobileNetV2 compiled and exported as MobileNetV2_tvm.so
